In [1]:
import pandas as pd
from nltk import word_tokenize
from gensim.models import Word2Vec

twitter_df = pd.read_csv('processed_cyberbullying_tweets.csv')
tweets = twitter_df['processed_tweet_text'].astype(str).tolist()

tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in tweets]
model_w2v = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

model_w2v.wv['food']


vocab_size = len(model_w2v.wv.key_to_index)
vocab_size

twitter_df['word_embeddings'] = twitter_df.apply(lambda x : list(), axis=1)

for index, row in twitter_df.iterrows():
    for word in word_tokenize(str(row['processed_tweet_text']).lower()):
        row['word_embeddings'].append(model_w2v.wv[word])
    while len(row['word_embeddings']) < 50:
        row['word_embeddings'].append([0]*100)
    

len(twitter_df.loc[0, 'word_embeddings'])
len(twitter_df.loc[0, 'word_embeddings'][0])


model_w2v.wv['words']

from sklearn.model_selection import train_test_split

X = twitter_df['word_embeddings'].tolist()
y = twitter_df['cyberbullying_type'].tolist()

data_0 = twitter_df[twitter_df['cyberbullying_type'] == 0]
data_1 = twitter_df[twitter_df['cyberbullying_type'] == 1]

sampled_data_0 = data_0.sample(n=7000, random_state=42)
sampled_data_1 = data_1.sample(n=10000, random_state=42)

balanced_df = pd.concat([sampled_data_0, sampled_data_1])

X_dec = balanced_df['word_embeddings'].tolist()
y_dec = balanced_df['cyberbullying_type'].tolist()

# Shuffle the resulting DataFrame to mix the examples of 0s and 1s
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(X_dec, y_dec, test_size=0.3, random_state=42)

/Users/jackoehling/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming X_train, X_test are lists of lists of word embeddings, and y_train, y_test are labels
# Since the input data is already in the form of embeddings, we don't need an embedding layer


In [3]:

# Padding sequences to ensure uniform input size
max_length = max(max(len(seq) for seq in X_train), max(len(seq) for seq in X_test))
X_train_padded = pad_sequences(X_train, maxlen=max_length, padding='post', dtype='float32')
X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post', dtype='float32')


In [4]:

model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(max_length, 100)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))  # Use 'softmax' for multi-class classification

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use 'categorical_crossentropy' for multi-class classification

# Training the model
model.fit(X_train_padded, np.array(y_train), epochs=5, batch_size=64, validation_split=0.2)

# Evaluating the model
loss, accuracy = model.evaluate(X_test_padded, np.array(y_test))
print(f'Test Accuracy: {accuracy}')

/Users/jackoehling/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 50s 313ms/step - accuracy: 0.7307 - loss: 0.4891 - val_accuracy: 0.8122 - val_loss: 0.4014
Epoch 2/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 50s 335ms/step - accuracy: 0.8113 - loss: 0.3841 - val_accuracy: 0.8227 - val_loss: 0.3695
Epoch 3/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 49s 326ms/step - accuracy: 0.8206 - loss: 0.3656 - val_accuracy: 0.8189 - val_loss: 0.3646
Epoch 4/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 48s 324ms/step - accuracy: 0.8209 - loss: 0.3651 - val_accuracy: 0.8185 - val_loss: 0.3597
Epoch 5/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 48s 325ms/step - accuracy: 0.8325 - loss: 0.3549 - val_accuracy: 0.8235 - val_loss: 0.3640
160/160 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.8210 - loss: 0.3679
Test Accuracy: 0.8156862854957581


In [13]:
def predict_sentiment(sentence, model, word2vec_model):
    # Tokenize and convert to embeddings
    tokens = word_tokenize(sentence.lower())
    print(tokens)
    embeddings = [word2vec_model.wv[token] if token in word2vec_model.wv else np.zeros(100) for token in tokens]
    
    # Pad the sequence
    padded_embeddings = pad_sequences([embeddings], maxlen=max_length, padding='post', dtype='float32')
    print(padded_embeddings)
    # Predict
    prediction = model.predict(padded_embeddings)
    print(prediction)
    # Assuming binary classification with a sigmoid output layer
    predicted_label = 'Positive' if prediction[0][0] < 0.5 else 'Negative'
    
    return predicted_label, prediction[0][0]

# Example usage
test_sentence = ""
predicted_label, confidence = predict_sentiment(test_sentence, model, model_w2v)

print(f"Sentence: '{test_sentence}'")
print(f"Predicted Sentiment: {predicted_label} (Confidence: {confidence:.2f})")

['u', 'fucking', 'retard']
[[[ 0.24201079  1.7237465   0.83498514 ... -0.7772788   0.7335766
   -1.3415278 ]
  [-0.0582964   0.09134184  0.4940063  ... -1.5268118  -0.4378817
    0.568826  ]
  [-0.15569425  0.02701895  0.20419548 ... -0.1593883  -0.04941026
    0.17396231]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[[0.37599513]]
Sentence: 'u fucking retard'
Predicted Sentiment: Positive (Confidence: 0.38)
